In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [13]:
lista_ligas = [
    "https://fbref.com/en/comps/12/2021-2022/2021-2022-La-Liga-Stats",
    "https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats",
    "https://fbref.com/en/comps/11/2021-2022/2021-2022-Serie-A-Stats",
    "https://fbref.com/en/comps/13/2021-2022/2021-2022-Ligue-1-Stats",
    "https://fbref.com/en/comps/20/2021-2022/2021-2022-Bundesliga-Stats",
]

years = list(range(2021, 2015, -1))
all_matches = []
log_error = []
log = []
for year in years:
    for indice, liga in enumerate(lista_ligas):
        data = requests.get(liga)
        soup = BeautifulSoup(data.text)

        try:
            standings_table = soup.select("table.stats_table")[0]
        except Exception as e:
            log_error.append(f"error:{e},{lista_ligas[indice]},{team_url}")
            continue

        links = [l.get("href") for l in standings_table.find_all("a")]
        links = [l for l in links if "/squads/" in l]
        team_urls = [f"https://fbref.com{l}" for l in links]

        previous_season = soup.select("a.prev")[0].get("href")
        print(lista_ligas[indice])

        time.sleep(1)
        for team_url in team_urls:
            print(team_url)
            team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
            data = requests.get(team_url)
            time.sleep(1)
            try:
                matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
                matches = matches[
                    [
                        "Date",
                        "Time",
                        "Comp",
                        "Round",
                        "Venue",
                        "Result",
                        "GF",
                        "GA",
                        "Opponent",
                        "Poss",
                        "Formation",
                    ]
                ]
                soup = BeautifulSoup(data.text)
                links = [l.get("href") for l in soup.find_all("a")]
                time.sleep(1)
            except Exception as e:
                log_error.append(f"error:{e},{lista_ligas[indice]},{team_url}")
                continue

            try:
                links_shooting = [l for l in links if l and "all_comps/shooting/" in l]
                data_shooting = requests.get(f"https://fbref.com{links_shooting[0]}")
                shooting = pd.read_html(data_shooting.text, match="Shooting")[0]
                shooting.columns = shooting.columns.droplevel()
                time.sleep(1)
            except Exception as e:
                log_error.append(f"error:{e},{lista_ligas[indice]},{team_url}")
                continue

            try:
                links_misc = [l for l in links if l and "all_comps/misc/" in l]
                data_misc = requests.get(f"https://fbref.com{links_misc[0]}")
                defensive = pd.read_html(data_misc.text, match="Miscellaneous Stats")[0]
                defensive.columns = defensive.columns.droplevel()
                time.sleep(1)

            except Exception as e:
                continue

            try:
                team_data = matches.merge(defensive[["Date", "TklW", "Int"]], on="Date")
            except ValueError:
                continue

            try:
                team_data = team_data.merge(
                    shooting[["Date", "Sh", "SoT", "Dist", "PK", "PKatt", "xG"]],
                    on="Date",
                )
            except Exception as e:
                team_data = team_data.merge(
                    shooting[["Date", "Sh", "SoT", "Dist", "PK", "PKatt"]], on="Date"
                )
                continue

            team_data["Season"] = year
            team_data["Team"] = team_name
            all_matches.append(team_data)
            time.sleep(2)
            pass

        lista_ligas[indice] = f"https://fbref.com{previous_season}"
        print("cambie a " + lista_ligas[indice])


https://fbref.com/en/comps/12/2021-2022/2021-2022-La-Liga-Stats
https://fbref.com/en/squads/53a2f082/2021-2022/Real-Madrid-Stats
https://fbref.com/en/squads/206d90db/2021-2022/Barcelona-Stats
https://fbref.com/en/squads/db3b9613/2021-2022/Atletico-Madrid-Stats


KeyboardInterrupt: 

In [ ]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df = match_df[['date', 'time', 'season', 'team', 'comp', 'round', 'venue', 'result', 'gf', 'ga',
       'opponent', 'poss', 'formation', 'tklw', 'int', 'sh', 'sot', 'dist',
       'pk', 'pkatt', 'xg']]
match_df.to_csv('matches.csv')

In [ ]:
match_df.tail()

,date,time,season,team,comp,round,venue,result,gf,ga,...,poss,formation,tklw,int,sh,sot,dist,pk,pkatt,xg
38,2018-04-14,15:30,2017,Koln,Bundesliga,Matchweek 30,Away,L,1,2,...,46.0,4-1-4-1,19.0,10.0,13.0,2.0,14.3,0.0,0.0,1.6
39,2018-04-22,18:00,2017,Koln,Bundesliga,Matchweek 31,Home,D,2,2,...,58.0,3-4-3,14.0,6.0,11.0,3.0,19.7,0.0,0.0,0.8
40,2018-04-28,15:30,2017,Koln,Bundesliga,Matchweek 32,Away,L,2,3,...,57.0,3-4-3,12.0,8.0,12.0,2.0,17.1,0.0,0.0,1.2
41,2018-05-05,15:30,2017,Koln,Bundesliga,Matchweek 33,Home,L,1,3,...,34.0,4-1-4-1,13.0,9.0,11.0,3.0,13.7,0.0,0.0,1.8
42,2018-05-12,15:30,2017,Koln,Bundesliga,Matchweek 34,Away,L,1,4,...,59.0,4-1-2-1-2◆,8.0,9.0,7.0,1.0,17.5,0.0,0.0,1.2


In [ ]:
all_matches[-1]

,Date,Time,Comp,Round,Venue,Result,GF,GA,Opponent,Poss,...,TklW,Int,Sh,SoT,Dist,PK,PKatt,xG,Season,Team
0,2017-08-12,15:30,DFB-Pokal,First round,Away,W,5,0,Leher,81,...,11,17,28,10,NaN,1,1,NaN,2017,Koln
1,2017-08-20,18:00,Bundesliga,Matchweek 1,Away,L,0,1,M'Gladbach,47,...,16,14,13,2,14.0,0,0,1.1,2017,Koln
2,2017-08-25,20:30,Bundesliga,Matchweek 2,Home,L,1,3,Hamburger SV,70,...,13,9,20,6,20.0,0,0,2.1,2017,Koln
3,2017-09-09,15:30,Bundesliga,Matchweek 3,Away,L,0,3,Augsburg,66,...,12,17,13,3,19.3,0,0,0.7,2017,Koln
4,2017-09-14,21:05,Europa Lg,Group stage,Away,L,1,3,eng Arsenal,29,...,15,13,10,4,25.9,0,0,0.4,2017,Koln
5,2017-09-17,18:00,Bundesliga,Matchweek 4,Away,L,0,5,Dortmund,27,...,14,20,6,2,18.9,0,0,0.7,2017,Koln
6,2017-09-20,18:30,Bundesliga,Matchweek 5,Home,L,0,1,Eint Frankfurt,55,...,13,15,17,7,18.6,0,0,2.1,2017,Koln
7,2017-09-24,15:30,Bundesliga,Matchweek 6,Away,D,0,0,Hannover 96,34,...,15,11,6,1,16.8,0,0,1.0,2017,Koln
8,2017-09-28,19:00,Europa Lg,Group stage,Home,L,0,1,rs Red Star,67,...,13,6,19,5,17.0,0,0,1.7,2017,Koln
9,2017-10-01,18:00,Bundesliga,Matchweek 7,Home,L,1,2,RB Leipzig,48,...,24,13,17,6,18.5,0,0,1.2,2017,Koln
